In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.model_selection import train_test_split

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf

import keras
import keras.backend as K

from tqdm import tqdm

from concurrent.futures import ThreadPoolExecutor

%matplotlib inline

2025-05-25 18:28:22.865134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748197703.146286      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748197703.231774      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 2. Image Loading method

In [2]:
def load_samples_dataframe(path: str):
    images_folder = os.path.join(path, "images")

    samples_df = pd.read_csv(
        os.path.join(path, "market_dataset_xy.txt"),
        sep=" ",
        header=None,
        names=["file path", "_", "datetime", "vel_y", "vel_x"]
    )

    samples_df["file path"] = (
        samples_df["file path"]
        .apply(lambda image_name: os.path.join(path, "images", image_name))
    )
    
    return samples_df

In [3]:
def make_dataset(samples_dataframe: pd.DataFrame):
    images = samples_dataframe["file path"].to_numpy()
    labels = {
        "vel_x": samples_dataframe["vel_x"].to_numpy(),
        "vel_y": samples_dataframe["vel_y"].to_numpy(),
    }

    return tf.data.Dataset.from_tensor_slices((images, labels))

In [4]:
def get_image_loader():
    @tf.function
    def load_image(image_path: str, label: dict):
        # Load grayscale image
        image = tf.io.read_file(image_path)
        image = tf.image.decode_image(image, channels=1, expand_animations=False)
        image = tf.image.convert_image_dtype(image, tf.float32)
    
        return image, label

    return load_image

In [5]:
def get_image_preprocessor(resolution: list, model_name: str):
    width, height, channels = resolution
    
    @tf.function
    def preprocess_image(image: tf.Tensor, label: dict):
        # Resizing
        image = tf.image.resize(image, [width, height])

        # Repeat grayscale image to match expected number of channels
        if channels != 1:
            image = tf.repeat(image, repeats=channels, axis=-1)

        # Model relative rescaling (if necessary)
        if model_name in ["MobileNet", "MobileNetV2"]:
            image = (image * 2.0) - 1.0

        elif model_name in ["ResNet50", "ResNet50V2", "ResNet18", "ResNet34"]:
            # Apply mean/std normalization per ImageNet
            mean = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
            std = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)
            image = (image - mean) / std

        return image, label

    return preprocess_image

# 3. Data augmentation method

In [6]:
@tf.function
def augment(image, label):
    # Brightness and contrast
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    
    # Flip horizontally
    do_flip = tf.random.uniform([]) > 0.5
    image = tf.cond(do_flip, lambda: tf.image.flip_left_right(image), lambda: image)
    
    vel_x = tf.cond(do_flip, lambda: -label["vel_x"], lambda: label["vel_x"])
    vel_y = label["vel_y"]
    
    # Optionally add noise
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.025)
    image = image + noise
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, {"vel_x": vel_x, "vel_y": vel_y}

# 4. Regression metrics

In [7]:
@tf.function
def rmse(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, 1])
    
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

@tf.function
def eva(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, 1])

    numerator = tf.math.reduce_variance(y_true - y_pred)
    denominator = tf.math.reduce_variance(y_true) + tf.keras.backend.epsilon()
    
    return 1.0 - numerator / denominator

@tf.function
def r2_score(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, 1])
    
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    
    return 1 - (ss_res / (ss_tot + K.epsilon()))

# 4. ResNet 8/18/34 construction

In [8]:
def resnet_block(x, filters, stride=1, use_projection=False):
    shortcut = x

    x = keras.layers.Conv2D(filters, 3, strides=stride, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)

    x = keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)

    if use_projection:
        shortcut = keras.layers.Conv2D(filters, 1, strides=stride, use_bias=False)(shortcut)
        shortcut = keras.layers.BatchNormalization()(shortcut)

    x = keras.layers.Add()([x, shortcut])
    x = keras.layers.ReLU()(x)
    return x

In [9]:
def ResNet8_DroNet(input_shape=(200, 200, 1), dropout_rate=0.3):
    inputs = keras.Input(shape=input_shape)

    x = keras.layers.Conv2D(32, 5, strides=2, padding='same', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)

    # Residual Blocks
    x = resnet_block(x, 32)
    x = resnet_block(x, 32)

    x = resnet_block(x, 64, stride=2, use_projection=True)
    x = resnet_block(x, 64)

    x = resnet_block(x, 128, stride=2, use_projection=True)
    x = resnet_block(x, 128)

    return keras.models.Model(inputs, x, name="ResNet8_DroNet")

In [10]:
def ResNet18(input_shape=(224, 224, 3)):
    inputs = keras.Input(shape=input_shape)
    
    x = keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Conv2_x
    x = resnet_block(x, 64)
    x = resnet_block(x, 64)

    # Conv3_x
    x = resnet_block(x, 128, stride=2, use_projection=True)
    x = resnet_block(x, 128)

    # Conv4_x
    x = resnet_block(x, 256, stride=2, use_projection=True)
    x = resnet_block(x, 256)

    # Conv5_x
    x = resnet_block(x, 512, stride=2, use_projection=True)
    x = resnet_block(x, 512)

    return keras.models.Model(inputs, x, name="ResNet18_backbone")


In [11]:
def ResNet34(input_shape=(224, 224, 3)):
    inputs = keras.Input(shape=input_shape)
    
    x = keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Conv2_x (3 blocks)
    for _ in range(3):
        x = resnet_block(x, 64)

    # Conv3_x (4 blocks)
    x = resnet_block(x, 128, stride=2, use_projection=True)
    for _ in range(3):
        x = resnet_block(x, 128)

    # Conv4_x (6 blocks)
    x = resnet_block(x, 256, stride=2, use_projection=True)
    for _ in range(5):
        x = resnet_block(x, 256)

    # Conv5_x (3 blocks)
    x = resnet_block(x, 512, stride=2, use_projection=True)
    for _ in range(2):
        x = resnet_block(x, 512)

    return keras.models.Model(inputs, x, name="ResNet34_backbone")


# 5. Funtion that instantiates the models

In [12]:
def instantiate_backbone(backbone_name):
    if backbone_name == "ResNet-8":
        return ResNet8_DroNet(input_shape=(200, 200, 1))
    
    if backbone_name == "ResNet-18":
        return ResNet18(input_shape=(224, 224, 3))
        
    if backbone_name == "ResNet-34":
        return ResNet34(input_shape=(224, 224, 3))

    if backbone_name == "ResNet-50":
        return keras.applications.ResNet50(
            include_top=False,
            input_shape=(224, 224, 3),
            weights=None,
            pooling=None,
        )

    if backbone_name == "ResNet-50V2":
        return keras.applications.ResNet50V2(
            include_top=False,
            input_shape=(224, 224, 3),
            weights=None,
            pooling=None,
        )

    if backbone_name == "MobileNet":
        return keras.applications.MobileNet(
            include_top=False,
            input_shape=(224, 224, 3),
            weights=None,
            pooling=None,
            alpha=1.0,
        )

    if backbone_name == "MobileNetV2":
        return keras.applications.MobileNetV2(
            include_top=False,
            input_shape=(224, 224, 3),
            weights=None,
            pooling=None,
            alpha=1.0,
        )

    if backbone_name == "EfficientNetB0":
        return keras.applications.EfficientNetB0(
            include_top=False,
            input_shape=(224, 224, 3),
            weights=None,
            pooling=None,
        )

# 6. Head descriptor 

In [13]:
def add_head(backbone, dense_layers_descriptions):
    x = keras.layers.GlobalAveragePooling2D(name="global_pool")(backbone.output)
    x = keras.layers.Dropout(0.3, name="dropout")(x)

    for i, (nodes, use_batch_normalization, use_dropout) in enumerate(dense_layers_descriptions):
        use_bias = not use_batch_normalization
        
        x = keras.layers.Dense(nodes, use_bias=use_bias, activation="linear", name=f"dense_{i}")(x)

        if use_batch_normalization:
            x = keras.layers.BatchNormalization(name=f"batchnorm_{i}")(x)

        x = keras.layers.ReLU(name=f"relu_{i}")(x)
        
        if use_dropout:
            x = keras.layers.Dropout(0.3, name=f"dropout_{i}")(x)
    
    output_x = keras.layers.Dense(1, name="vel_x")(x)
    output_y = keras.layers.Dense(1, name="vel_y")(x)

    return {"vel_x": output_x, "vel_y": output_y}

# Tests

In [14]:
def compile_model(model):
    model.compile(
        loss={
            "vel_x": "mean_squared_error",
            "vel_y": "mean_squared_error",
        },
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        metrics={
            "vel_x": [rmse, eva, r2_score],
            "vel_y": [rmse, eva, r2_score],
        },
    )

    return model

In [15]:
def train_model(model: keras.Model, train_dataset: tf.data.Dataset, validation_dataset: tf.data.Dataset):
    history_1 = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        verbose=1,
        epochs=40,
    )

    history_2 = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=200,
        verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=45,
                min_delta=1e-4,
                mode="min",
                restore_best_weights=True,
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor="val_loss",
                patience=15,
                factor=0.5,
                min_lr=1e-7,
                mode="min"
            ),
        ],
    )

    full_history = pd.concat([
        pd.DataFrame(history_1.history),
        pd.DataFrame(history_2.history),
    ]).reset_index(drop=True)

    return model, full_history

In [16]:
models_descriptions = [
    {
        "name": "ResNet-8-base",
        "backbone": "ResNet-8",
        "head_description": [],
    },
    {
        "name": "ResNet-8-600A",
        "backbone": "ResNet-8",
        "head_description": [
            (64, False, False),
        ],
    },
    {
        "name": "ResNet-8-600B",
        "backbone": "ResNet-8",
        "head_description": [
            (64, True, False),
        ],
    },
    {
        "backbone": "ResNet-18",
        "head_description": [],
    },
    {
        "backbone": "ResNet-34",
        "head_description": [],
    },
    {
        "backbone": "ResNet-50",
        "head_description": [],
    },
    {
        "backbone": "ResNet-50V2",
        "head_description": [],
    },
    {
        "backbone": "MobileNet",
        "head_description": [],
    },
    {
        "backbone": "MobileNetV2",
        "head_description": [],
    },
    {
        "backbone": "EfficientNetB0",
        "head_description": [],
    },
]

ROUNDS = 3

# Folder structure creation
if os.path.exists("/kaggle/working/results"):
    shutil.rmtree("/kaggle/working/results")

os.mkdir("results")
for round_number in range(1, ROUNDS + 1):
    os.mkdir(f"results/round_{round_number}")
    for model_description in models_descriptions:
        name = model_description.setdefault("name", model_description["backbone"])
        os.mkdir(f"results/round_{round_number}/{name}")

# List of round metrics
metrics_per_round = [{} for _ in range(ROUNDS)]

for round_number in range(1, ROUNDS + 1):
    # Cache of datasets
    dataset_cache = {}
    
    for model_description in models_descriptions:
        keras.backend.clear_session()
        
        # Model instantiation
        name = model_description.setdefault("name", model_description["backbone"])
        
        backbone = instantiate_backbone(model_description["backbone"])
        outputs = add_head(backbone, model_description["head_description"])
        
        model = keras.models.Model(inputs=backbone.inputs, outputs=outputs, name=name)
        model = compile_model(model)

        # Construction of the train, validation and test datasets
        samples_df = load_samples_dataframe("/kaggle/input/marketplace-navigation-dataset/dataset")

        train_df, temp_df = train_test_split(samples_df, train_size=0.6, random_state=round_number)
        test_df, val_df = train_test_split(temp_df, train_size=0.5, random_state=round_number)

        input_shape = model.input_shape[-3:]
        image_loader = get_image_loader()
        preprocess = get_image_preprocessor(input_shape, name)

        # Create dataset if not exists
        if input_shape not in dataset_cache:
            train_dataset = (
                make_dataset(train_df)
                .shuffle(buffer_size=len(train_df), seed=round_number)
                .map(image_loader, num_parallel_calls=tf.data.AUTOTUNE)
                .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
                .cache()
                .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
                .batch(64)
                .prefetch(tf.data.AUTOTUNE)
            )
    
            validation_dataset = (
                make_dataset(val_df)
                .map(image_loader, num_parallel_calls=tf.data.AUTOTUNE)
                .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
                .cache()
                .batch(64)
                .prefetch(tf.data.AUTOTUNE)
            )
    
            test_dataset = (
                make_dataset(test_df)
                .map(image_loader, num_parallel_calls=tf.data.AUTOTUNE)
                .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
                .cache()
                .batch(64)
                .prefetch(tf.data.AUTOTUNE)
            )

            dataset_cache[input_shape] = (train_dataset, validation_dataset, test_dataset,)

        # Load datasets
        train_dataset, validation_dataset, test_dataset = dataset_cache[input_shape]
        
        # Training
        print(f"=========== Model {name} | Round {round_number} ==========")
        model, history = train_model(model, train_dataset, validation_dataset)

        # Evaluation
        metrics_per_round[round_number - 1][name] = model.evaluate(test_dataset, return_dict=True)
        
        model.save(f"/kaggle/working/results/round_{round_number}/{name}/model.keras")
        history.to_csv(f"/kaggle/working/results/round_{round_number}/{name}/history.csv")

I0000 00:00:1748197716.347780      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


=========== Model ResNet-8-base | Round 1 ==========
Epoch 1/40


I0000 00:00:1748197732.611572      57 service.cc:148] XLA service 0x7bc868003390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748197732.612660      57 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1748197734.588987      57 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/22 ━━━━━━━━━━━━━━━━━━━━ 8:34 24s/step - loss: 4.3674 - vel_x_eva: -1.3047 - vel_x_loss: 2.9398 - vel_x_r2_score: -6.3689 - vel_x_rmse: 1.7146 - vel_y_eva: -1.3108 - vel_y_loss: 1.4276 - vel_y_r2_score: -1.3209 - vel_y_rmse: 1.1948

I0000 00:00:1748197743.635957      57 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

22/22 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - loss: 2.5378 - vel_x_eva: -0.8100 - vel_x_loss: 0.7320 - vel_x_r2_score: -0.8907 - vel_x_rmse: 0.8487 - vel_y_eva: -0.5943 - vel_y_loss: 1.8033 - vel_y_r2_score: -1.4987 - vel_y_rmse: 1.3167 - val_loss: 2.3182 - val_vel_x_eva: 2.2352e-07 - val_vel_x_loss: 0.5259 - val_vel_x_r2_score: -0.0717 - val_vel_x_rmse: 0.7113 - val_vel_y_eva: 1.2666e-07 - val_vel_y_loss: 1.7163 - val_vel_y_r2_score: -1.3723 - val_vel_y_rmse: 1.3064
Epoch 2/40
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 164ms/step - loss: 1.6436 - vel_x_eva: -0.4419 - vel_x_loss: 0.5890 - vel_x_r2_score: -0.4770 - vel_x_rmse: 0.7587 - vel_y_eva: -0.3441 - vel_y_loss: 1.0545 - vel_y_r2_score: -0.4368 - vel_y_rmse: 1.0226 - val_loss: 2.2929 - val_vel_x_eva: 2.5332e-07 - val_vel_x_loss: 0.5276 - val_vel_x_r2_score: -0.0768 - val_vel_x_rmse: 0.7126 - val_vel_y_eva: 2.3097e-07 - val_vel_y_loss: 1.6897 - val_vel_y_r2_score: -1.3351 - val_vel_y_rmse: 1.2962
Epoch 3/40
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step -

E0000 00:00:1748205431.372602      60 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748205431.566149      60 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748205432.111020      60 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748205432.318870      60 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748205432.710034      60 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

21/22 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - loss: 3.5442 - vel_x_eva: -2.4916 - vel_x_loss: 1.1847 - vel_x_r2_score: -2.5322 - vel_x_rmse: 1.0272 - vel_y_eva: -0.8733 - vel_y_loss: 2.3595 - vel_y_r2_score: -2.1822 - vel_y_rmse: 1.4674

E0000 00:00:1748205464.980751      57 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748205465.165824      57 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748205465.603680      57 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748205465.812503      57 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748205466.166151      57 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

22/22 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - loss: 3.4583 - vel_x_eva: -2.4189 - vel_x_loss: 1.1650 - vel_x_r2_score: -2.4592 - vel_x_rmse: 1.0191 - vel_y_eva: -0.8433 - vel_y_loss: 2.2901 - vel_y_r2_score: -2.0889 - vel_y_rmse: 1.4438 - val_loss: 1.7383 - val_vel_x_eva: 2.2352e-07 - val_vel_x_loss: 0.5144 - val_vel_x_r2_score: -0.0334 - val_vel_x_rmse: 0.7019 - val_vel_y_eva: 1.8626e-07 - val_vel_y_loss: 1.1499 - val_vel_y_r2_score: -0.5801 - val_vel_y_rmse: 1.0681
Epoch 2/40
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 217ms/step - loss: 1.7025 - vel_x_eva: -1.0195 - vel_x_loss: 0.7992 - vel_x_r2_score: -1.0387 - vel_x_rmse: 0.8739 - vel_y_eva: -0.1657 - vel_y_loss: 0.9026 - vel_y_r2_score: -0.2262 - vel_y_rmse: 0.9462 - val_loss: 1.5769 - val_vel_x_eva: 2.5332e-07 - val_vel_x_loss: 0.5262 - val_vel_x_r2_score: -0.0725 - val_vel_x_rmse: 0.7115 - val_vel_y_eva: 1.6391e-07 - val_vel_y_loss: 0.9810 - val_vel_y_r2_score: -0.3449 - val_vel_y_rmse: 0.9861
Epoch 3/40
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 215ms/step 

In [17]:
metrics_per_round_dataframes = [pd.DataFrame(metrics).T for metrics in metrics_per_round]

final_results = sum(metrics_per_round_dataframes) / len(metrics_per_round_dataframes)
final_results.to_csv("/kaggle/working/results/final_metrics.csv")

for round_number, metrics in enumerate(metrics_per_round_dataframes, 1):
    metrics.to_csv(f"/kaggle/working/results/round_{round_number}/round_metrics.csv")